In [ ]:
from convertors.convert_to_json import *

In [ ]:
global In_Silico_pattern
In_Silico_pattern = re.compile(r"in.silico|insilico|predicted|theoretical|Annotation.level.3", flags=re.IGNORECASE)

def in_filename(filename):
    """
    :param filename: The name of the file to be checked for the presence of the string "MSMS_Public" and matching a specific pattern.
    :return: Returns True if the filename does not contain "MSMS_Public" and matches a specific pattern defined by the In_Silico_pattern. Returns False otherwise.

    """
    if "MSMS_Public" not in filename:
        if re.search(In_Silico_pattern, filename):
            return True

    return False

def normalize_predicted(metadata_dict):
    """
    Normalize the predicted field in the given metadata dictionary.

    :param metadata_dict: A dictionary containing metadata information.
    :return: The updated metadata dictionary with the normalized predicted field.
    """
    comment_field = metadata_dict["COMMENT"]
    predicted = metadata_dict["PREDICTED"]
    filename = metadata_dict["FILENAME"]

    if re.search(In_Silico_pattern, comment_field) or predicted == "true" or in_filename(filename):
        metadata_dict["PREDICTED"] = "true"
        return metadata_dict
    else:
        metadata_dict["PREDICTED"] = "false"
        return metadata_dict

In [ ]:
def convert_keys(dict_list):
    """
    Convert keys in metadata_dict based on the provided keys_dict and keys_list.

    :param metadata_dict: A dictionary containing metadata information.
    :return: A dictionary with converted keys based on the provided keys_dict and keys_list.
    """
    output = []
    for metadata_dict in dict_list:
        converted = {keys_dict[key.lower()]: val for key, val in metadata_dict.items() if key.lower() in keys_dict and keys_dict[key.lower()] in keys_list}
    
        converted.update({key: "" for key in keys_list if key not in converted})
        output.append(converted)

    return output

In [ ]:
original_db_path = r"C:\Users\Axel\Documents\PYTHON\FragHub\INPUT"

In [ ]:
FINAL_MSP, FINAL_XML, FINAL_CSV, FINAL_JSON, FINAL_MGF = convert_to_json(original_db_path)

In [ ]:
FINAL_MSP = convert_keys(FINAL_MSP)
FINAL_XML = convert_keys(FINAL_XML)
FINAL_CSV = convert_keys(FINAL_CSV)
FINAL_JSON = convert_keys(FINAL_JSON)
FINAL_MGF = convert_keys(FINAL_MGF)

In [ ]:
print(len(FINAL_MSP)+len(FINAL_XML)+len(FINAL_CSV)+len(FINAL_JSON) + len(FINAL_MGF))

In [ ]:
compteur = 0
for i in range(len(FINAL_MSP)):
    FINAL_MSP[i]["PREDICTED"] = ""
    FINAL_MSP[i] = normalize_predicted(FINAL_MSP[i])
compteur = 0
for i in range(len(FINAL_XML)):
    FINAL_XML[i]["PREDICTED"] = ""
    FINAL_XML[i] = normalize_predicted(FINAL_XML[i])
compteur = 0
for i in range(len(FINAL_CSV)):
    FINAL_CSV[i]["PREDICTED"] = ""
    FINAL_CSV[i] = normalize_predicted(FINAL_CSV[i])
compteur = 0
for i in range(len(FINAL_JSON)):
    FINAL_JSON[i]["PREDICTED"] = ""
    FINAL_JSON[i] = normalize_predicted(FINAL_JSON[i])
compteur = 0
for i in range(len(FINAL_MGF)):
    FINAL_MGF[i]["PREDICTED"] = ""
    FINAL_MGF[i] = normalize_predicted(FINAL_MGF[i])

In [ ]:
msp_df = pd.DataFrame(FINAL_MSP)
del FINAL_MSP
xml_df = pd.DataFrame(FINAL_XML)
del FINAL_XML
csv_df = pd.DataFrame(FINAL_CSV)
del FINAL_CSV
json_df = pd.DataFrame(FINAL_JSON)
del FINAL_JSON
mgf_df = pd.DataFrame(FINAL_MGF)
del FINAL_MGF
DF = pd.concat([msp_df, xml_df, csv_df, json_df, mgf_df])
del msp_df, xml_df, csv_df, json_df

In [ ]:
LC = DF[~DF['INSTRUMENTTYPE'].str.contains('GC|EI', case=False)]

In [ ]:
LC_exp = LC[LC['PREDICTED'] == "false"]

In [ ]:
print(len(LC_exp))

In [ ]:
print(len(DF['INCHI'].unique()))